In [12]:
import os
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import pretty_midi as pm
import ast

In [2]:
ls

commuMetaData.ipynb   criandoFuncaoPerdas+Latentes.ipynb  recons_umap.ipynb
compute_losses.ipynb  estudandoPerdas.ipynb               umapPlots.ipynb
createChdTex.ipynb    process_features.ipynb


In [3]:
ls ../

NotebooksVAESymTex/     dataSet/                   midiDataTest/
README.md               dl_modules/                midiTo4bin.py
__pycache__/            draftCode/                 model_param/
base_model/             estudandoPerdasPrompts.md  processMidiPath.py
calc_latent_loss.py     features/                  scripts_python.py
chordData.py            interface.py               test_npz_integrity.py
commu00001_4bin.npz     latent_features/           testeFunctions/
commuTestNPZ/           losses/                    utilProcessing.py
compute_single_loss.py  losses909ToPlot/           viewNPZ.py


In [4]:
ls ../midiDataTest/

commu00001.mid  commu00002.mid  commu00003.mid  commu_meta.csv


In [5]:
metaDataCOMMU = pd.read_csv("../midiDataTest/commu_meta.csv")

In [6]:
metaDataCOMMU

,Unnamed: 0,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,0,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid,8,120,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001
1,1,cmajor,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Dm'...",mid_low,8,80,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002
2,2,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid_high,8,150,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003
3,3,cmajor,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Gm'...",mid,8,110,cinematic,pad,choir,standard,4/4,45,46,train,commu00004
4,4,aminor,"[['Am', 'Am', 'Am', 'Am', 'Em', 'Em', 'Em', 'E...",mid_low,4,60,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11139,11139,cmajor,"[['Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7',...",mid_high,8,110,cinematic,pad,synth_pad,standard,4/4,83,84,val,commu11140
11140,11140,cmajor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid_low,8,110,cinematic,riff,acoustic_guitar,standard,4/4,51,55,val,commu11141
11141,11141,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",very_low,8,145,cinematic,bass,string_ensemble,standard,4/4,109,112,val,commu11142
11142,11142,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",low,8,140,cinematic,accompaniment,string_cello,standard,4/4,55,115,val,commu11143


In [7]:
print(metaDataCOMMU["time_signature"].unique())

['4/4' '6/8' '3/4']


In [9]:
print(metaDataCOMMU["sample_rhythm"].value_counts())

sample_rhythm
standard    10751
triplet       393
Name: count, dtype: int64


In [23]:
input_signature = metaDataCOMMU["time_signature"]
print(input_signature)

0        4/4
1        4/4
2        4/4
3        4/4
4        4/4
        ... 
11139    4/4
11140    4/4
11141    4/4
11142    4/4
11143    4/4
Name: time_signature, Length: 11144, dtype: object


In [26]:
numerator = metaDataCOMMU["time_signature"].str.split("/").str[0].astype(int)
print(numerator)

0        4
1        4
2        4
3        4
4        4
        ..
11139    4
11140    4
11141    4
11142    4
11143    4
Name: time_signature, Length: 11144, dtype: int64


In [24]:
numerator = int(input_signature.split("/")[0])

AttributeError: 'Series' object has no attribute 'split'

In [27]:
def filtro_chords_por_beat(df):
    return df[
        df.apply(lambda row: len(ast.literal_eval(row["chord_progressions"])[0]) / numerator / row["num_measures"] == 2,
                 axis=1)
    ]

In [29]:
def filtro_chords_por_beat(df):
    return df[
        df.apply(lambda row: len(ast.literal_eval(row["chord_progressions"])[0]) /
                            int(row["time_signature"].split("/")[0]) /
                            row["num_measures"] == 2,
                 axis=1)
    ]

In [30]:
filtro_chords_por_beat(metaDataCOMMU)

,Unnamed: 0,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,0,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid,8,120,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001
1,1,cmajor,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Dm'...",mid_low,8,80,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002
2,2,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid_high,8,150,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003
3,3,cmajor,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Gm'...",mid,8,110,cinematic,pad,choir,standard,4/4,45,46,train,commu00004
4,4,aminor,"[['Am', 'Am', 'Am', 'Am', 'Em', 'Em', 'Em', 'E...",mid_low,4,60,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11139,11139,cmajor,"[['Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7',...",mid_high,8,110,cinematic,pad,synth_pad,standard,4/4,83,84,val,commu11140
11140,11140,cmajor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid_low,8,110,cinematic,riff,acoustic_guitar,standard,4/4,51,55,val,commu11141
11141,11141,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",very_low,8,145,cinematic,bass,string_ensemble,standard,4/4,109,112,val,commu11142
11142,11142,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",low,8,140,cinematic,accompaniment,string_cello,standard,4/4,55,115,val,commu11143
